# PageRank

In this notebook, we will use both NetworkX and cuGraph to compute the PageRank of each vertex in our test dataset.  The NetworkX and cuGraph processes will be interleaved so that each step can be compared.

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Created:   08/13/2019
* Updated:   01/23/2020

RAPIDS Versions: 0.12.0a    

Test Hardware

* GV100 32G, CUDA 10.0


## Introduction
Pagerank is measure of the relative importance, also called centrality, of a vertex based on the relative importance of it's neighbors.  PageRank was developed by Google and is (was) used to rank it's search results. PageRank uses the connectivity information of a graph to rank the importance of each vertex. 

See [Wikipedia](https://en.wikipedia.org/wiki/PageRank) for more details on the algorithm.

To compute the Pagerank scores for a graph in cuGraph we use:<br>

**cugraph.pagerank(G,alpha=0.85, max_iter=100, tol=1.0e-5)**
* __G__: cugraph.Graph object
* __alpha__: float, The damping factor represents the probability to follow an outgoing edge. default is 0.85
* __max_iter__: int, The maximum number of iterations before an answer is returned. This can be used to limit the execution time and do an early exit before the solver reaches the convergence tolerance. If this value is lower or equal to 0 cuGraph will use the default value, which is 100
* __tol__: float, Set the tolerance the approximation, this parameter should be a small magnitude value. The lower the tolerance the better the approximation. If this value is 0.0f, cuGraph will use the default value which is 0.00001. Setting too small a tolerance can lead to non-convergence due to numerical roundoff. Usually values between 0.01 and 0.00001 are acceptable.

Returns:
* __df__: a cudf.DataFrame object with two columns:
    * df['vertex']: The vertex identifier for the vertex
    * df['pagerank']: The pagerank score for the vertex


## cuGraph Notice 
The current version of cuGraph has some limitations:

* Vertex IDs need to be 32-bit integers.
* Vertex IDs are expected to be contiguous integers starting from 0.

cuGraph provides the renumber function to mitigate this problem. Input vertex IDs for the renumber function can be either 32-bit or 64-bit integers, can be non-contiguous, and can start from an arbitrary number. The renumber function maps the provided input vertex IDs to 32-bit contiguous integers starting from 0. cuGraph still requires the renumbered vertex IDs to be representable in 32-bit integers. These limitations are being addressed and will be fixed soon.    

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


### Prep

In [1]:
# The notebook compares cuGraph to NetworkX,  
# therefore there some additional non-RAPIDS python libraries need to be installed. 
# Please run this cell if you need the additional libraries
!pip install networkx
!pip install scipy

In [2]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

In [3]:
# NetworkX libraries
import networkx as nx
from scipy.io import mmread

### Some Prep

In [4]:
# define the parameters 
max_iter = 100  # The maximum number of iterations
tol = 0.00001   # tolerance
alpha = 0.85    # alpha

In [5]:
# Define the path to the test data  
datafile='../data/karate-data.csv'

---
# NetworkX

In [6]:
# Read the data, this also created a NetworkX Graph 
file = open(datafile, 'rb')
Gnx = nx.read_edgelist(file)

In [7]:
pr_nx = nx.pagerank(Gnx, alpha=alpha, max_iter=max_iter, tol=tol)

In [8]:
pr_nx

{'1': 0.09701602879679112,
 '2': 0.05288352140273713,
 '3': 0.05707824474239912,
 '4': 0.03586215186028656,
 '5': 0.021982948286246907,
 '6': 0.0291187010552997,
 '7': 0.0291187010552997,
 '8': 0.024490182143018978,
 '9': 0.02976223516027387,
 '11': 0.021982948286246907,
 '12': 0.009564948925264983,
 '13': 0.014645330577424499,
 '14': 0.02953411440759793,
 '18': 0.014558421109725606,
 '20': 0.01960235337430456,
 '22': 0.014558421109725606,
 '32': 0.037151296649534996,
 '31': 0.024585898152929067,
 '10': 0.014307076351595165,
 '28': 0.025635977707690095,
 '29': 0.019571497200047343,
 '33': 0.07169586521384512,
 '17': 0.01678918865030142,
 '34': 0.10092357454636647,
 '15': 0.01453184130992307,
 '16': 0.01453184130992307,
 '19': 0.01453184130992307,
 '21': 0.01453184130992307,
 '23': 0.01453184130992307,
 '24': 0.03151619714292456,
 '26': 0.021005086910429464,
 '30': 0.026283644274280444,
 '25': 0.021074699693936733,
 '27': 0.015041538663860602}

Running NetworkX is that easy.  
Let's seet how that compares to cuGraph

----

# cuGraph

### Read in the data - GPU
cuGraph depends on cuDF for data loading and the initial Dataframe creation

The data file contains an edge list, which represents the connection of a vertex to another.  The `source` to `destination` pairs is in what is known as Coordinate Format (COO).  In this test case, the data is just two columns.  However a third, `weight`, column is also possible

In [9]:
# Read the data  
gdf = cudf.read_csv(datafile, names=["src", "dst"], delimiter='\t', dtype=["int32", "int32"] )

### Create a Graph 

In [10]:
# create a Graph using the source (src) and destination (dst) vertex pairs from the Dataframe 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst')

### Call the PageRank algorithm

In [11]:
# Call cugraph.pagerank to get the pagerank scores
gdf_page = cugraph.pagerank(G)

_It was that easy!_  
Compared to NetworkX, the cuGraph data loading might have been more steps, but using cuDF allows for a wider range of data to be loaded. 


----

Let's now look at the results

In [12]:
# Find the most important vertex using the scores
# This methods should only be used for small graph
bestScore = gdf_page['pagerank'][0]
bestVert = gdf_page['vertex'][0]

for i in range(len(gdf_page)):
    if gdf_page['pagerank'][i] > bestScore:
        bestScore = gdf_page['pagerank'][i]
        bestVert = gdf_page['vertex'][i]
        
print("Best vertex is " + str(bestVert) + " with score of " + str(bestScore))

Best vertex is 34 with score of 0.10091735


The top PageRank vertex and socre match what was found by NetworkX

In [13]:
# A better way to do that would be to find the max and then use that values in a query
pr_max = gdf_page['pagerank'].max()

In [14]:
def print_pagerank_threshold(_df, t=0) :
    filtered = _df.query('pagerank >= @t')
    
    for i in range(len(filtered)):
        print("Best vertex is " + str(filtered['vertex'][i]) + 
            " with score of " + str(filtered['pagerank'][i]))              

In [15]:
print_pagerank_threshold(gdf_page, pr_max)

Best vertex is 34 with score of 0.10091735


----

a PageRank score of _0.10047_ is very low, which can be an indication that there is no more central vertex than any other.  Rather than just looking at the top score, let's look at the top three vertices and see if there are any insights that can be inferred.  

Since this is a very small graph, let's just sort and get the first three records

In [16]:
sort_pr = gdf_page.sort_values('pagerank', ascending=False)

In [17]:
sort_pr.head(3)

,vertex,pagerank
33,34,0.100917
0,1,0.096999
32,33,0.071692


Going back and looking at the graph with the top three vertices highlighted (illustration below) it is easy to see that the top scoring vertices also appear to be the vertices with the most connections.   
Let's look at sorted list of degrees (since the graph is undirected and symmetrized, the out degree is the same as the in degree)

In [18]:
d = G.degrees()

In [19]:
d.sort_values('out_degree', ascending=False).head(4)

,vertex,in_degree,out_degree
33,34,17,17
0,1,16,16
32,33,12,12
2,3,10,10


<img src="../img/zachary_graph_pagerank.png" width="600">

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___